In [1]:
from lstm import LSTM
from tokenizer import Vocabulary
from dense import Dense
from embedding import EmbeddingLayer
import numpy as np

In [6]:
class LSTMSequential:
    def __init__(self):
        self.network = {}
        self.caches = []
        self.states = []
        self.kernel_gradients = []
        self.recurrent_kernel_gradients = []
        self.state_gradients = []
        self.embedding_weights = None
        self.dense_weights = None
        
    def add(self, layer):
        self.network[layer.name] = layer
        
    def _init_hidden(self):
        hidden = self.network['LSTM'].hidden_dim
        state = {'h':np.zeros((hidden,)), 'c':np.zeros((hidden,))}
    
        return state
    
    def single_forward(self, data):
        init_state = self._init_hidden()
        
        lstm_inp = self.network['Embedding'].predict(data)
        cache, state = self.network['LSTM'].forward(lstm_inp, 0, init_state)
        final = self.network['Dense'].forward(state['h'])
        
        return lstm_inp, state['h'], final
    
    def single_backward(self, prediction, actual):
        self.network['LSTM'].backward(prediction, actual, 0)
        
model = LSTMSequential()

model.add(EmbeddingLayer(vocab_size=2855, hidden_dim=20))
model.add(LSTM(units=100, features=20, seq_length=25))
model.add(Dense(2855))

test = np.array([   8,   11,  273,    2,   98,   27,  464,    6,  341,   74,   15,
        416,   17,    0, 1070,    1,  342,  127,   50,  132,   53,  588,
          4,   19,  837])

lstm_inp, lstm_out, final = model.single_forward(test)
# model.single_backward(final, 61)

In [7]:
lstm_inp.shape, lstm_out.shape, final.shape

((25, 20), (25, 100), (25, 2855))